In [1]:
import xgboost as xgb

In [2]:
import sklearn

In [9]:
#pip install -U scikit-learn==1.0.2

Note: you may need to restart the kernel to use updated packages.


In [4]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.0.2.


In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('DS.csv')

In [7]:
df['Selling_Price'] = df['Selling_Price'].str.replace(',','').astype(float)

In [8]:
df['Selling_Price'] = df['Selling_Price'].astype(int)

In [9]:
df['Present_Price'] = df['Present_Price'].str.replace(',','').astype(float)

In [10]:
df['Present_Price'] = df['Present_Price'].astype(int)

In [11]:
df

,Car_Name,Manufacturer,Year,Kms_Driven,Fuel_Type,Seller_Type,Transmission,No_Of_Owners,Present_Price,Selling_Price,Insurance
0,Maruti Suzuki Dzire Ldi,Maruti,2011,100000,Diesel,Individual,Manual,1,750000,310000,Yes
1,Maruti Suzuki Dzire Ldi,Maruti,2011,100000,Diesel,Dealer,Manual,1,750000,350000,Yes
2,Maruti Suzuki Dzire vdi,Maruti,2011,25000,Diesel,Individual,Manual,1,850000,350000,Yes
3,Maruti Suzuki Dzire Zdi,Maruti,2011,50000,Diesel,Dealer,Manual,2,900000,400000,Yes
4,Maruti Suzuki Dzire Ldi,Maruti,2011,100000,Diesel,Individual,Manual,1,750000,299000,No
...,...,...,...,...,...,...,...,...,...,...,...
1463,Toyota Fortuner 2.8 4X2,Toyota,2021,5698,Diesel,Individual,Automatic,1,4384000,3950000,Yes
1464,Toyota Fortuner 2.8 4X4,Toyota,2021,20452,Diesel,Individual,Manual,1,4522000,4300000,Yes
1465,Toyota Fortuner 2.8 4X4,Toyota,2021,4800,Diesel,Individual,Automatic,1,4792000,4500000,Yes
1466,Toyota Legender,Toyota,2021,9000,Diesel,Individual,Manual,1,4896000,4600000,Yes


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as plt
%matplotlib inline
plt.style.use('ggplot')
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [13]:
df_X=df.drop(['Selling_Price'],axis=1)
df_y=df['Selling_Price']

In [14]:
df_X_train,df_X_test,df_y_train,df_y_test=train_test_split(df_X,df_y,test_size=0.1,random_state=100)

In [15]:
ohe=OneHotEncoder()
ohe.fit(df[['Car_Name','Manufacturer','Fuel_Type','Seller_Type','Transmission','Insurance']])

OneHotEncoder()

In [16]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['Car_Name','Manufacturer','Fuel_Type','Seller_Type','Transmission','Insurance']),
                                    remainder='passthrough')

In [17]:
xgb_reg = xgb.XGBRegressor()

In [18]:
xg_pipe=make_pipeline(column_trans,xgb_reg)

In [19]:
xg_pipe.fit(df_X_train,df_y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Honda Accord 2.4', 'Honda Accord Elegance',
       'Honda Accord V6 Inspire', 'Honda Amaze iDtec S',
       'Honda Amaze iDtec VX', 'Honda Amaze iDtech SCVT',
       'Honda Amaze iDtech SX', 'Honda Amaze iDtech VCVT',
       'Honda Amaze iDtech VX', 'Honda Amaze iVtec VX...
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.300000012, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_est

In [36]:
y_predXG=xg_pipe.predict(df_X_test)

In [20]:
scores=[]
for i in range(50):
    df_X_train,df_X_test,df_y_train,df_y_test=train_test_split(df_X,df_y,test_size=0.1,random_state=i)
    
    pipe=make_pipeline(column_trans,xgb_reg)
    pipe.fit(df_X_train,df_y_train)
    y_pred=pipe.predict(df_X_test)
    scores.append(r2_score(df_y_test,y_pred))

In [21]:
df_X_train,df_X_test,df_y_train,df_y_test=train_test_split(df_X,df_y,test_size=0.1,random_state=7)
    
pipe=make_pipeline(column_trans,xgb_reg)
pipe.fit(df_X_train,df_y_train)
y_pred=pipe.predict(df_X_test)
#print(scores.append(r2_score(df_y_test,y_pred)))
r2_score(df_y_test,y_pred)

0.9878885086038214

In [22]:
pipe.predict(pd.DataFrame(columns=['Car_Name','Manufacturer','Year','Kms_Driven','Fuel_Type','Seller_Type','Transmission','No_Of_Owners','Present_Price','Insurance'],
                          
                          data=np.array(['Toyota Fortuner 2.8 4X4','Toyota',2021,20452,'Diesel','Dealer','Manual',1,4522000,'Yes']).reshape(1,10)))


array([4292114.5], dtype=float32)

In [23]:
 np.argmax(scores)

7

In [24]:
pipe.predict(pd.DataFrame(columns=['Car_Name','Manufacturer','Year','Kms_Driven','Fuel_Type','Seller_Type','Transmission','No_Of_Owners','Present_Price','Insurance'],
                          
                          data=np.array(['Maruti Suzuki Alto Lxi','Maruti',2015,80000,'Petrol','Individual','Automatic',1,380000,'Yes']).reshape(1,10)))


array([267459.16], dtype=float32)

In [25]:
scores[np.argmax(scores)]

0.9878885086038214

In [26]:
import pickle

In [27]:
pickle.dump(pipe,open('dfXGBRegressorNew.pkl','wb'))

In [28]:
op=pipe.predict(pd.DataFrame(columns=['Car_Name','Manufacturer','Year','Kms_Driven','Fuel_Type','Seller_Type','Transmission','No_Of_Owners','Present_Price','Insurance'],
                          
                          data=np.array(['Toyota Legender 4X4','Toyota',20,15000,'Diesel','Individual','Automatic',1,5336000,'Yes']).reshape(1,10)))
op = abs(op)
print(op)


[3991098.2]


In [49]:
df.head()

,Car_Name,Manufacturer,Year,Kms_Driven,Fuel_Type,Seller_Type,Transmission,No_Of_Owners,Present_Price,Selling_Price,Insurance
0,Maruti Suzuki Dzire Ldi,Maruti,2011,100000,Diesel,Individual,Manual,1,750000,310000,Yes
1,Maruti Suzuki Dzire Ldi,Maruti,2011,100000,Diesel,Dealer,Manual,1,750000,350000,Yes
2,Maruti Suzuki Dzire vdi,Maruti,2011,25000,Diesel,Individual,Manual,1,850000,350000,Yes
3,Maruti Suzuki Dzire Zdi,Maruti,2011,50000,Diesel,Dealer,Manual,2,900000,400000,Yes
4,Maruti Suzuki Dzire Ldi,Maruti,2011,100000,Diesel,Individual,Manual,1,750000,299000,No


In [51]:
op=pipe.predict(pd.DataFrame(columns=['Car_Name','Manufacturer','Year','Kms_Driven','Fuel_Type','Seller_Type','Transmission','No_Of_Owners','Present_Price','Insurance'],
                          
                          data=np.array(['Maruti Suzuki Dzire Ldi','Maruti',2011,100000,'Diesel','Individual','Manual',1,750000,'No']).reshape(1,10)))
op = abs(op)
print(op)


[310311.8]


In [37]:
r2_score(df_y_test,y_predXG)

0.8079084422802794

In [28]:
def objective(trial,data=df_X,target=df_y):
    df_X_train,df_X_test,df_y_train,df_y_test=train_test_split(df_X,df_y,test_size=0.1,random_state=100)
    params = {
        'tree_method':'gpu_hist',
        'lambda':trial.suggest_loguniform('lambda',1e-4,10.0),
        'alpha':trial.suggest_loguniform('alpha',1e-4,10.0),
        'colsample_bytree':trial.suggest_categorical('colsample_bytree',[0.1,0.2,0.3,.5,.6,.7,.8,.9,1]),
        'subsample':trial.suggest_categorical('subsample',[0.1,0.2,0.3,.5,.6,.7,.8,.9,1]),
        'learning_rate':trial.suggest_categorical('learning_rate',[0.00001,0.0003,0.008,0.02,0.01,1,10]),
        'n_estimators':30000,
        'max_depth':trial.suggest_categorical('max_depth',[3,4,5,6,7,8,9,10,11,12]),
        'random_state':trial.suggest_categorical('random_state',[10,20,30,40,50,60,70,80,90,1510,3250]),
        'min_child_weight':trial.suggest_int('min_child_weight',1,2000),
        'verbosity':3
    }
    xgb_reg_model=xgb.XGBRegressor(**params)
    xgb_pipe=make_pipeline(column_trans,xgb_reg_model)
    xgb_pipe.fit(df_X_train,df_y_train)
    xgb_pred=xgb_pipe.predict(df_X_test)
    r2_score(df_y_test,xgb_pred)
    

In [20]:
#import optuna

In [21]:
find_param=optuna.create_study(direction='minimize')
find_param.optimize(objective,n_trials=10)
find_param.best_trial.params

NameError: name 'optuna' is not defined